# **ANN DENGAN FEEDBACK**

### **IMPORT LIBRARY**

In [76]:
import sys
print(sys.executable)
print(sys.version)

c:\Users\Hanna\AppData\Local\Programs\Python\Python313\python.exe
3.13.2 (tags/v3.13.2:4f8bb39, Feb  4 2025, 15:23:48) [MSC v.1942 64 bit (AMD64)]


In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

### **DATASET**

**Model matematis *plant***:

$$
y(k)= \frac{1}{[1+y(k-1)]^2} + 0,25x(k) - 0,3x(k-1)
$$
</br></nr>
**Info database**:</br>
x(t) = generate random -1 s.d. 1</br>
y(t) = model plant
</br></br>
**Dimensi data**:</br>
30000 $\times$ 2

**Buat dataset**

In [78]:
def plant_data(N):
    # buat nilai x
    np.random.seed(42)
    x_val = np.random.uniform(-1, 1, N)

    # buat nilai y
    y_val = np.zeros(N)
    y_val[0] = 0.0

    for k in range(1, N):
        y_val[k] = 1/(1 + y_val[k-1]**2) + 0.25*x_val[k] - 0.3*x_val[k-1]

    dataset = pd.DataFrame({'x': x_val, 'y': y_val})
    return dataset

# Jumlah data
N = 30004

dk = plant_data(N)
dk

,x,y
0,-0.250920,0.000000
1,0.901429,1.300633
2,0.463988,0.217088
3,0.197317,0.865127
4,-0.687963,0.340751
...,...,...
29999,-0.485204,0.424156
30000,0.276289,1.062157
30001,-0.081415,0.366645
30002,0.928997,1.138175


In [79]:
df_value1 = pd.DataFrame({
    'Min_value': dk.min(),
    'Max_value': dk.max()
})

df_value1

,Min_value,Max_value
x,-0.999977,0.999850
y,-0.230966,1.543344


**Normalisasi Data**

Normalisasi data input dan output ke [-1,1]

In [80]:
def min_max_scaling(df):
    df_scaled = df.copy()
    for column in df_scaled.columns:
        min_val = df_scaled[column].min()
        max_val = df_scaled[column].max()
        df_scaled[column] = (2 * (df_scaled[column] - min_val) / (max_val - min_val)) - 1
    return df_scaled

dk_scaled = min_max_scaling(dk)
dk_scaled

,x,y
0,-0.250878,-0.739655
1,0.901570,0.726417
2,0.464092,-0.494954
3,0.197398,0.235514
4,-0.687959,-0.355561
...,...,...
29999,-0.485183,-0.261547
30000,0.276377,0.457607
30001,-0.081359,-0.326373
30002,0.929141,0.543294


In [81]:
df_value2 = pd.DataFrame({
    'Min_value': dk_scaled.min(),
    'Max_value': dk_scaled.max()
})

df_value2

,Min_value,Max_value
x,-1.0,1.0
y,-1.0,1.0


**Model Data**

In [82]:
def lagged_dataset(df, cols, lag):
    df_new = pd.DataFrame()
    for col in cols:
        for i in range(lag+1):
            if i == 0:
                new_col_name = f'{col}(t)'
            else:
                new_col_name = f'{col}(t-{i})'

            # geser nilai kolom ke bawah sebanyak i
            df_new[new_col_name] = df[col].shift(i)
    
    # drop baris dengan nilai NaN
    df_new = df_new.dropna().reset_index(drop=True)

    return df_new

df_lag = lagged_dataset(dk_scaled, cols=['x', 'y'], lag=4)
df_lag.head()

,x(t),x(t-1),x(t-2),x(t-3),x(t-4),y(t),y(t-1),y(t-2),y(t-3),y(t-4)
0,-0.687959,0.197398,0.464092,0.901570,-0.250878,-0.355561,0.235514,-0.494954,0.726417,-0.739655
1,-0.688007,-0.687959,0.197398,0.464092,0.901570,0.309039,-0.355561,0.235514,-0.494954,0.726417
2,-0.883846,-0.688007,-0.687959,0.197398,0.464092,-0.151846,0.309039,-0.355561,0.235514,-0.494954
3,0.732479,-0.883846,-0.688007,-0.687959,0.197398,0.651803,-0.151846,0.309039,-0.355561,0.235514
4,0.202311,0.732479,-0.883846,-0.688007,-0.687959,-0.483698,0.651803,-0.151846,0.309039,-0.355561


### **EDA** (Exploratory Data Analysis)

**Struktur data**

In [83]:
print("Info Dataset")
print("Ukuran dataset lag:", df_lag.shape, "\n")
df_lag.info()

Info Dataset
Ukuran dataset lag: (30000, 10) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x(t)    30000 non-null  float64
 1   x(t-1)  30000 non-null  float64
 2   x(t-2)  30000 non-null  float64
 3   x(t-3)  30000 non-null  float64
 4   x(t-4)  30000 non-null  float64
 5   y(t)    30000 non-null  float64
 6   y(t-1)  30000 non-null  float64
 7   y(t-2)  30000 non-null  float64
 8   y(t-3)  30000 non-null  float64
 9   y(t-4)  30000 non-null  float64
dtypes: float64(10)
memory usage: 2.3 MB


**Deskripsi data**

In [84]:
df_lag.describe().T

,count,mean,std,min,25%,50%,75%,max
x(t),30000.0,-0.000802,0.575900,-1.0,-0.497520,0.000553,0.495643,1.0
x(t-1),30000.0,-0.000776,0.575892,-1.0,-0.497475,0.000607,0.495643,1.0
x(t-2),30000.0,-0.000792,0.575873,-1.0,-0.497475,0.000607,0.495608,1.0
x(t-3),30000.0,-0.000759,0.575897,-1.0,-0.497475,0.000647,0.495643,1.0
x(t-4),30000.0,-0.000777,0.575896,-1.0,-0.497475,0.000607,0.495643,1.0
y(t),30000.0,0.032763,0.376475,-1.0,-0.243049,0.035925,0.310335,1.0
y(t-1),30000.0,0.032795,0.376451,-1.0,-0.243038,0.035940,0.310335,1.0
y(t-2),30000.0,0.032760,0.376452,-1.0,-0.243049,0.035925,0.310288,1.0
y(t-3),30000.0,0.032796,0.376468,-1.0,-0.243038,0.035940,0.310335,1.0
y(t-4),30000.0,0.032756,0.376486,-1.0,-0.243049,0.035925,0.310288,1.0


**Cek missing values**

In [85]:
df_lag.isna().sum()

x(t)      0
x(t-1)    0
x(t-2)    0
x(t-3)    0
x(t-4)    0
y(t)      0
y(t-1)    0
y(t-2)    0
y(t-3)    0
y(t-4)    0
dtype: int64

**Cek duplikasi**

In [86]:
df_lag.duplicated().sum()

np.int64(0)

### **PRE-PROCESSING DATA**

**Split Data Input dan Target**

In [87]:
X = df_lag[['x(t)', 'x(t-1)', 'x(t-2)', 'x(t-3)', 'x(t-4)', 'y(t-1)', 'y(t-2)', 'y(t-3)', 'y(t-4)']]
Y = df_lag[['y(t)']]

print("Input set shape :", X.shape)
print("Target set shape:", Y.shape)

Input set shape : (30000, 9)
Target set shape: (30000, 1)


**Split Model Dataset**

X = [X(t), X(t-1), X(t-2), X(t-3), X(t-4), Y(t-1), Y(t-2), Y(t-3), Y(t-4)]</br>
Y = [Y(t)]</br>

**Training** : 15000 dataset</br>
**Validasi** : 10000 (5000 data training akhir + 5000 dataset)</br>
**Testing**  : 10000 dataset


In [88]:
def split_dataset(X, Y):
    assert len(X) == len(Y), "Panjang X dan Y harus sama"
    assert len(X) == 30000, "Dataset harus 30.000 sample sesuai aturan"

    X_train = X[:15000]
    Y_train = Y[:15000]

    X_val = np.concatenate([X_train[-5000:], X[15000:20000]])
    Y_val = np.concatenate([Y_train[-5000:], Y[15000:20000]])

    X_test = X[20000:]
    Y_test = Y[20000:]

    return X_train, Y_train, X_val, Y_val, X_test, Y_test

X_train, Y_train, X_val, Y_val, X_test, Y_test = split_dataset(X, Y)

print("Training shape  :", X_train.shape, Y_train.shape)
print("Validation shape:", X_val.shape, Y_val.shape)
print("Test shape      :", X_test.shape, Y_test.shape)

Training shape  : (15000, 9) (15000, 1)
Validation shape: (10000, 9) (10000, 1)
Test shape      : (10000, 9) (10000, 1)


# STAGE 1: DEEP LEARNING

### **Neural Network Model**